# Tensorflow Object Detection for Drone
## Detect Drone with Image using OpenCV
### Setup paths and directories

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # suppress TF logging
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR')           # suppress TF logging

In [3]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [17]:
IMAGE_PATHS = ('C:/Users/timjr/.keras/datasets/Drone1.jpg', 'C:/Users/timjr/.keras/datasets/Drone2.jpg', 'C:/Users/timjr/.keras/datasets/Drone3.jpg')

### Download the Labels
The next step is to download the labels file (`.pbtxt`) which contains a list of strings used to add the correct label to each detection (like a person). Since the pre-trained model we will use has been trained on the COCO (Common Object in Context) dataset, we will need to download the labels file corresponding to this dataset, named `mscoco_label_map.pbtxt`.

In [9]:
PATH_TO_LABELS = 'C:/Users/timjr/Documents/TensorFlow/workspace/training_demo/exported-models/my_ssd_mobilenet_v2_fpnlite/drone_label_map.pbtxt'

### Load the Model
Next, we will load the downloaded model.

In [10]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

In [11]:
PATH_TO_SAVED_MODEL = 'C:/Users/timjr/Documents/TensorFlow/workspace/training_demo/exported-models/my_ssd_mobilenet_v2_fpnlite/saved_model'

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Loading model took {elapsed_time:.2} seconds.')

Loading model...Done! Loading model took 9.0 seconds.


### Load Label Map Data (For Plotting)
Label maps correspond index numbers to category names, so that when our convolution network predicts 5, we know that this corresponds to airplane. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

In [12]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [13]:
category_index

{1: {'id': 1, 'name': 'Drone'}}

### Putting Everything Together
Finally we will load an image, run it through the detection model, and visualize the detection results, including the keypoints.

Note - This will take a long time (several minutes) the first time it is run due to `tf.function`'s trace-compilation.

Some things to try:
- Modify some of the input images and see if detection still works. Some simple things to try out here (just uncomment the relevant portions of code) include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).
- Print out detections[‘detection_boxes’] and try to match the box locations to the boxes in the image. Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
- Set `min_score_thresh` to other values (between 0 and 1) to allow more detections in or to filter out more detections.

In [14]:
import numpy as np
from PIL import Image    # Pillow (Python image library)
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')    # Suprress Matplotlib warnings

In [15]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

In [21]:
%matplotlib
for image_path in IMAGE_PATHS:
    print(f'Running inference for {image_path}... ', end='')
    
    image_np = load_image_into_numpy_array(image_path)
    
    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)
    
    # The input needs to be a tensor, convert it using 'tf.conver_to_tensor'.
    input_tensor = tf.convert_to_tensor(image_np)
    
    # The model expects a batch of images, so add an axis with 'tf.newaxis'.
    input_tensor = input_tensor[tf.newaxis, ...]
    
    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    
    # All outputs are batches tensors
    # Convert to numpy arrays and take index [0] to remove the batch dimension
    # We're only interested in the first num_detections
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    # detection_classes should be ints 
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()

# sphinx_gallery_thumbnail_number = 2

Using matplotlib backend: Qt5Agg
Running inference for C:/Users/timjr/.keras/datasets/Drone1.jpg... Done
Running inference for C:/Users/timjr/.keras/datasets/Drone2.jpg... Done
Running inference for C:/Users/timjr/.keras/datasets/Drone3.jpg... Done
